In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [ ]:
import fnmatch
train = []
tes = []
val = []

for root,dirs,files in os.walk('./3fold_out4/train'):
    if files:
        lst = os.listdir(root)
        png = fnmatch.filter(lst,'*.png')
        if png:
            for i in png:
                train.append((root+'/'+i, int(i[3])-1))
for root,dirs,files in os.walk('./3fold_out4/test'):
      if files:
        lst = os.listdir(root)
        png = fnmatch.filter(lst,'*.png')
        if png:
            for i in png:
                tes.append((root+'/'+i, int(i[3])-1))
for root,dirs,files in os.walk('./3fold_out4/val'):
      if files:
        lst = os.listdir(root)
        png = fnmatch.filter(lst,'*.png')
        if png:
            for i in png:
                val.append((root+'/'+i, int(i[3])-1))

In [3]:
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import matplotlib.pyplot as plt

In [4]:
trn = tes+val
test = train

In [5]:
class_num = len(trn)

In [6]:
class_num

338

In [7]:
class MyDataset(Dataset):
    def __init__(self, image_path, transform = None):
        self.image_list = image_path
        self.len = len(self.image_list)
        self.transform = transform
    def __getitem__(self, index):
        fn, label = self.image_list[index]
        img = Image.open(fn)
        label = np.array(label,dtype = np.float64)
        if self.transform is not None:
            img = self.transform(img) 
        return img, index, label
    def __len__(self):
        return self.len

Transform = transforms.Compose([
    transforms.RandomRotation(degrees=15),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    #transforms.CenterCrop(224),
    transforms.Resize(128),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ])
Lable_trasfrom = transforms.Compose([
    transforms.ToTensor()
    ])

In [8]:
train_data = MyDataset(trn, transform = Transform)
test_data = MyDataset(test, transform = Transform)
train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=32, shuffle=False)

In [9]:
import torchvision.models as models
from torch import nn,optim

In [10]:
class EmbeddingNet(torch.nn.Module):
    def __init__(self, embedding_size=128):
        super(EmbeddingNet, self).__init__()
        self.densenet = models.resnet18(pretrained=True)
        self.densenet.fc = torch.nn.Sequential(
            torch.nn.Linear(512,embedding_size),
            #torch.nn.ReLU(),
            #torch.nn.Dropout(0.6),
            #torch.nn.Linear(256,embedding_size)
        )
        
    def forward(self, x):
        embedding = self.densenet(x)
        norm = embedding.norm(p=2, dim=1, keepdim=True)
        embedding = embedding.div(norm)
        return embedding
    
    def get_embedding(self, x):
        return self.forward(x)

In [11]:
net = EmbeddingNet()

class class_net(torch.nn.Module):
    def __init__(self, embedding_net):
        super(class_net, self).__init__()
        self.net = embedding_net
        self.classes =  nn.Sequential(
            torch.nn.ReLU(),
            torch.nn.Linear(128,class_num),
            torch.nn.LogSoftmax(dim=1)
        )
        
    def forward(self, x):
        embedding = self.net(x)
        labels = self.classes(embedding)
        return labels
    
    def get_embedding(self, x):
        return self.forward(x)
    
my_resnet = class_net(net)

In [12]:
my_resnet = my_resnet.to('cuda:0')

In [13]:
#loss_func = FocalLoss()
#loss_func = NPairLoss()
loss_func = nn.NLLLoss()
lr = 1e-2
#optimizer = optim.Adam(my_resnet.parameters(), lr=lr)
#optimizer = optim.Adam(my_resnet.parameters(), lr=lr)
optimizer = optim.SGD(my_resnet.parameters(),
                                lr=lr,
                                momentum=0.9,
                                weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=300, gamma=0.1, last_epoch=-1)

In [14]:
import time

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
history = []
best_acc = 0.0
best_epoch = 0
best_each = ''

best_loss = 100.0

In [15]:
for epoch in range(500):
    epoch_start = time.time()
    print("Epoch: {}/{}".format(epoch+1, 500))
    my_resnet.train()
    train_loss = 0.0
    valid_loss = 0.0
    train_acc = 0.0
    valid_acc = 0.0
    for i,(data,label,index) in enumerate(train_loader):
        data = data.to(device)
        label = label.to(device)
        label = label.long()
        optimizer.zero_grad()
        outputs = my_resnet(data)
        loss = loss_func(outputs,label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        ret, predictions = torch.max(outputs.data, 1)
        correct_counts = predictions.eq(label.data.view_as(predictions))
        acc = torch.mean(correct_counts.type(torch.FloatTensor))
        train_acc += acc.item() * data.size(0)
    with torch.no_grad():
        my_resnet.eval()
        #correct = list(0. for i in range(5))
        #total = list(0. for i in range(5))
        #correct = list(0. for i in range(2))
        #total = list(0. for i in range(2))
        for j, (inputs, labels, indexs) in enumerate(test_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            labels = labels.long()
            indexs = indexs.to(device)
            indexs = indexs.long()

            outputs = my_resnet(inputs)

            #loss = loss_func(outputs, labels)
            #valid_loss += loss.item() * inputs.size(0)
            ret, predictions = torch.max(outputs.data, 1)
            for label_idx in range(len(labels)):
                predictions[label_idx] = trn[predictions[label_idx]][1]
            correct_counts = predictions.eq(indexs.data.view_as(predictions))
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            valid_acc += acc.item() * inputs.size(0)
            res = predictions == labels
            '''for label_idx in range(len(labels)):
                label_single = labels[label_idx]
                correct[label_single] += res[label_idx].item()
                total[label_single] += 1'''

    scheduler.step()
    avg_train_loss = train_loss/len(trn)
    avg_train_acc = train_acc/len(trn)

    avg_valid_loss = valid_loss/len(test)
    avg_valid_acc = valid_acc/len(test)

    #acc_str = 'Accuracy: %f'%(sum(correct)/sum(total))

    '''for acc_idx in range(5):
    #for acc_idx in range(2):
        try:
            acc = correct[acc_idx]/total[acc_idx]
        except:
            acc = 0
        finally:
            acc_str += ' classID:%d acc:%f '%(acc_idx+1, acc)'''

    history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])
    if best_acc <= avg_valid_acc:
        best_acc = avg_valid_acc
        best_epoch = epoch + 1
        torch.save(my_resnet.state_dict(),'./weight_inst/bestloss_train_338.pth')

    epoch_end = time.time()

    print("Epoch: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation: Loss: {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch+1, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))
    print("Best Accuracy for validation : {:.4f} at epoch {:03d}".format(best_acc, best_epoch))
    torch.save(my_resnet.state_dict(),'./weight_inst/lastloss_train_338.pth')
    #print(acc_str)#print("Best Acc's each class acc:",best_each)
print("Best Acc's each class acc:",best_each)

Epoch: 1/500
Epoch: 001, Training: Loss: 5.8263, Accuracy: 0.0000%, 
		Validation: Loss: 0.0000, Accuracy: 25.1656%, Time: 1.0347s
Best Accuracy for validation : 0.2517 at epoch 001
Epoch: 2/500
Epoch: 002, Training: Loss: 5.8203, Accuracy: 0.2959%, 
		Validation: Loss: 0.0000, Accuracy: 37.7483%, Time: 0.7857s
Best Accuracy for validation : 0.3775 at epoch 002
Epoch: 3/500
Epoch: 003, Training: Loss: 5.8164, Accuracy: 0.0000%, 
		Validation: Loss: 0.0000, Accuracy: 29.1391%, Time: 0.7556s
Best Accuracy for validation : 0.3775 at epoch 002
Epoch: 4/500
Epoch: 004, Training: Loss: 5.8155, Accuracy: 0.2959%, 
		Validation: Loss: 0.0000, Accuracy: 26.4901%, Time: 0.7652s
Best Accuracy for validation : 0.3775 at epoch 002
Epoch: 5/500
Epoch: 005, Training: Loss: 5.8091, Accuracy: 1.1834%, 
		Validation: Loss: 0.0000, Accuracy: 32.4503%, Time: 0.7625s
Best Accuracy for validation : 0.3775 at epoch 002
Epoch: 6/500
Epoch: 006, Training: Loss: 5.8058, Accuracy: 0.0000%, 
		Validation: Loss: 0